<a href="https://colab.research.google.com/github/botmaker00/Rex-Deploy/blob/main/rex_hk_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
  <img src="https://files.catbox.moe/4w80av.jpg" height="250"/>
  <h1>🔥 REX ENCODER BOT - HEROKU DEPLOYER 🔥</h1>
  <h3>Ab sirf 5 minute mein apna powerful encoder bot live karo!</h3>
  <b>Channel:</b> @RexBots_Official | <b>Support:</b> @rexbotschat | <b>Dev:</b> @botmaker00
</div>

In [ ]:
#@title 🔐 Step 1: Heroku Login
Heroku_Email = ""  #@param {type:"string"}
Heroku_API_Key = ""  #@param {type:"string"}
from IPython.display import clear_output, HTML
import os
!curl -s https://cli-assets.heroku.com/install.sh | sh
clear_output()
if not Heroku_Email or not Heroku_API_Key:
    display(HTML('<h3 style="color:red">❌ Heroku Email aur API Key enter karo!</h3>'))
else:
    netrc = f"machine api.heroku.com\n login {Heroku_Email}\n password {Heroku_API_Key}\n" + \
            f"machine git.heroku.com\n login {Heroku_Email}\n password {Heroku_API_Key}\n"
    with open(os.path.expanduser("~/.netrc"), "w") as f: f.write(netrc)
    os.chmod(os.path.expanduser("~/.netrc"), 0o600)
    !git config --global user.email {Heroku_Email}
    !git config --global user.name RexBot_User
    display(HTML('<h2 style="color:lime">✅ Login Successful!</h2>'))

In [ ]:
#@title 🚀 Step 2: Create Heroku App (Optional)
App_Names = ""  #@param {type:"string"}
Region = "eu"  #@param ["eu", "us"]
Team_Name = ""  #@param {type:"string"}
if App_Names.strip():
    team = f"--team {Team_Name}" if Team_Name else ""
    for app in App_Names.split():
        !heroku create {app.lower()} --region {Region} --stack container {team}
else:
    print("ℹ️ No new app created.")

In [ ]:
#@title ⚙️ Step 3: Config Setup
App_Name = "rexencoder"  #@param {type:"string"}
API_ID = 123  #@param {type:"integer"}
API_HASH = ""  #@param {type:"string"}
BOT_TOKEN = ""  #@param {type:"string"}
OWNER_ID = 123  #@param {type:"integer"}
SUDO_USERS = ""  #@param {type:"string"}
LOG_CHANNEL = -100  #@param {type:"integer"}
SESSION_NAME = "rexencoder"  #@param {type:"string"}
MONGO_URI = ""  #@param {type:"string"}
Gist_URL = ""  #@param {type:"string"}
Upload_Config_File = False  #@param {type:"boolean"}
import shutil
if os.path.exists(App_Name): shutil.rmtree(App_Name)
!git clone https://github.com/botmaker00/Rex-Deploy {App_Name}
%cd {App_Name}
!rm -f README.md *.ipynb
if Upload_Config_File:
    from google.colab import files
    up = files.upload()
    for f in up.keys(): os.rename(f, "config.env")
elif Gist_URL.strip():
    !curl -L -o config.env {Gist_URL}
else:
    with open("config.env", "w") as f:
        f.write(f"API_ID={API_ID}\nAPI_HASH='{API_HASH}'\nBOT_TOKEN='{BOT_TOKEN}'\nOWNER_ID={OWNER_ID}\nSUDO_USERS='{SUDO_USERS}'\nLOG_CHANNEL={LOG_CHANNEL}\nSESSION_NAME='{SESSION_NAME}'\nMONGO_URI='{MONGO_URI}'\n")
%cd ..
print(f"Config ready for {App_Name}")

In [ ]:
#@title 🚀 Step 4: Deploy Now
Deploy_Apps = "rexencoder"  #@param {type:"string"}
for app in Deploy_Apps.split():
    if os.path.isdir(app):
        %cd {app}
        !git add .
        !git commit -m Deploy --allow-empty
        !heroku git:remote -a {app} || true
        !git push heroku main -f
        %cd ..
        print(f"🎉 {app} deployed!")
    else:
        print(f"❌ Folder not found: {app}")

In [ ]:
#@title 📊 Step 5: View Logs
App_Logs = ""  #@param {type:"string"}
if App_Logs.strip(): !heroku logs -t -a {App_Logs}
else: print("Enter app name")

In [ ]:
#@title 🚪 Step 6: Logout
!heroku logout -i
print("Logged out!")